<a href="https://colab.research.google.com/github/yashtripya/ai_email_generator/blob/main/Gen_AI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("🔄 Loading the model. Please wait...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,
    device=0 if torch.cuda.is_available() else -1
)
print("✅ Model loaded successfully!")

def build_prompt(purpose, recipient, points, tone, sender):
    bullet_text = '\n'.join(f"- {point}" for point in points)
    return (
        f"Write a {tone} email to a {recipient} regarding: {purpose}.\n"
        f"Include the following key points:\n{bullet_text}\n"
        f"Start with a subject line, then write the body.\n"
        f"End the email with: \nRegards,\n{sender}\n\n"
        f"Subject:"
    )

def generate_text(prompt):
    output = generator(
        prompt,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    raw_text = output[0]["generated_text"].strip()
    return clean_output(raw_text, prompt)

def clean_output(text, prompt):
    if text.startswith(prompt):
        text = text[len(prompt):].strip()
    last_punc = max(text.rfind(p) for p in [".", "!", "?"])
    return text[:last_punc + 1].strip() if last_punc != -1 else text.strip()

def get_inputs():
    print("\n" + "=" * 60)
    print("📬 AI EMAIL GENERATOR".center(60))
    print("=" * 60)

    sender = input("✍️  Enter your name (Sender): ").strip()
    recipient = input("👤 Who is the recipient? (e.g., Manager, Friend): ").strip()
    purpose = input("📝 What is the purpose of the email? (e.g., job application, invite): ").strip()

    print("\n🧾 Enter bullet points you want included (type 'done' to finish):")
    points = []
    while True:
        item = input("➤ ").strip()
        if item.lower() == "done":
            break
        if item:
            points.append(item)

    print("\n🎯 Choose the tone of your email:\n1. Formal\n2. Informal")
    tone = "formal" if input("Select option (1 or 2): ").strip() == "1" else "informal"

    return purpose, recipient, points, tone, sender

def main():
    purpose, recipient, points, tone, sender = get_inputs()
    prompt = build_prompt(purpose, recipient, points, tone, sender)
    print("\n✨ Generating your email. Please wait...")
    result = generate_text(prompt)
    print("\n" + "=" * 60)
    print("📨 Generated Email:\n")
    print(result)

if __name__ == "__main__":
    main()


🔄 Loading the model. Please wait...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded successfully!

                    📬 AI EMAIL GENERATOR                    
✍️  Enter your name (Sender): yash
👤 Who is the recipient? (e.g., Manager, Friend): manager
📝 What is the purpose of the email? (e.g., job application, invite): leave application

🧾 Enter bullet points you want included (type 'done' to finish):
➤ need leave for 3 days
➤ to attend sister's wedding in hometown
➤ starting from 10th july
➤ done

🎯 Choose the tone of your email:
1. Formal
2. Informal
Select option (1 or 2): 1

✨ Generating your email. Please wait...

📨 Generated Email:

Request for Leave

Dear [Manager's Name],

I am writing to request a leave of absence for three days, starting from [date], to attend my sister's wedding in our hometown.

I am excited to attend this important occasion, which will be a great opportunity for me to spend time with family and friends.

The wedding is scheduled to take place on [date] at [time]. I will need to leave the office on [date] and return on [date